# UNIVERSITY OF SAN DIEGO - MS AAI
## Natural Language Processing and Generative AI
### Final Project - Team 1: Multi-Agent Financial Analysis System.
#### By Manikandan Perumal & Israel Romero Olvera
#### _________________________________________________
#### The purpose of this final project is to build a real-world financial analysis system powered by agentic AI, with the abilities of reasoning, planning, and acting based on the user's prompt. It will coordinate multiple specialized LLM agents to handle complex financial tasks end-to-end.
#### Our Agentic AI system was developed in a folder structure that can be found in our GitHub site: https://github.com/isralennon/AAI_520_Group_1/tree/main
#### For delivery purposes we've condensed all the code into this document, structured the following way:
#### **1. Tools** - this section contains the code in file /modules/tools.py which will perform basic RAG connections.
#### **2. Parser** - this section contains the code in file /modules/parser.py, which provides basic functionality to parse data in JSON format.
#### **3. Memory** - this section contains the code in file /modules/memory.py that handles the storage of ongoing knowledge, to provide a robust and efficient functionality.
#### **4. Agents** - this section contains the code in file /modules/subagents.py, designed to host the definitions of the main Agent class as well as our specialized subagents - the team of agents available to the main orchestrator. We developed the following team of agents:
#### - **News Researcher** - the specialist of finding financial news, using FinnHub.
#### - **Market Researcher** - the specialist of finding financial hard data like market trends, stock prices, etc.
#### - **Writer** - the specialist of taking all the information and preparing a polished answer for the user
#### **5. Main Orchestrator Agent** - this section contains the code in file /modules/agent.py and has the definition for the orchestrator agent, which develops the strategy and coordinates all subagents.
#### **6. Demo** - this section contains the code in our main notebook, /main.ipynb - our implementation file where we execute all the above with demonstration purposes.
#### _________________________________________________
### 1. TOOLS.
#### One of the four agent functions we'll implement is the usage of tools, which will be defined in this first section.

In [1]:
import dotenv
import os
#import modules.tools as tools
import yfinance as yf
import requests
import finnhub
from typing import Callable
from datetime import datetime, timedelta
from google import genai
import openai

# For privacy reasons, we'll store our token keys on a .env file, which we'll load here:
dotenv.load_dotenv(dotenv_path=".env")

# First, we'll define a generic Tool class, which will serve as a structure for all of our tools
class Tool:
    def __init__(self, name, function, description, api=None): # This is the initialization method of the class
        self.name = name # Placeholder for the name of the tool
        self.function = function # Placeholder for the code of the tool's function
        self.description = description # Placeholder for the description of the tool - very important since the agents will use this description to know what the tool does
        self.api = api  # Placeholder for API details when needed
        
    def to_dict(self): # The structure of each class will always be a standard dictionary object that can be easily interpreted by the Agents
        return {
            "name": self.name,
            "description": self.description,
            "api": self.api
        }
    
    def invoke(self, **kwargs): # This is the placeholder of the function for the tool, which will receive a variable number of parameters
        print(f"Invoking {self.name} with arguments {kwargs}")
        return self.function(**kwargs) # Returning the results of the function

# Next, we'll declare each individual tool as a class, inheriting from the generic class Tool above
class YahooFinance(Tool): # The first tool is YahooFinance, which will pull stock quotes for a given financial symbol, like AAPL for Apple
    def __init__(self):
        super().__init__(
            name="Yahoo Finance Stock Quote", # Name of the tool
            function=self.get_stock_quote_yahoo, # Pointing to the YahooFinance function below as this class's own function
            description="Get the latest stock quote for a given symbol from Yahoo Finance.", # Definition of the tool for our agents
            api="""{ ""symbol": "AAPL"}""", # Parameter sample for the agent to use when it uses this class
        )
    def get_stock_quote_yahoo(self, symbol: str, step: str='') -> dict: # This is the function that pulls the stock using YahooFinance API
        # Here we'll perform the call to YahooFinance to get the data from the specified symbol.
        ticker = yf.Ticker(symbol)
        # Then, we'll use the 'fast_info' method, which pulls basic financial information, including the price.
        try:
            info = ticker.fast_info # Pulling the information and parsing it to return it
            return {
                "symbol": symbol,
                "last_price": info["lastPrice"],
                "day_high": info["dayHigh"],
                "day_low": info["dayLow"],
                "previous_close": info["previousClose"]
            }
        except Exception as e: # Should there be any errors, we will print the error message instead and return an empty dictionary
            print(f"Yahoo Finance API error: {e}")
            return {}
#Now, we'll continue with the class that calls Financial Modeling Prep API
class FMP(Tool):
    def __init__(self,name:str,function:Callable=None,description:str=None,api:str=None,endPoint:str=None):
        super().__init__(name=name,function=self.execute if function==None else function,description=description,api=api)
        self.endpoint = endPoint if endPoint!=None else  os.getenv("FMP_Endpoint") # It reads the endpoint from our .env file
        self.apikey = os.getenv("FMP_API_KEY") # It also reads the API key from our .env file
    def execute(self, symbol: str) -> dict: # This is the function that pulls the stock data using FMP API
        params = { #These are the parameters for the API call in a dictionary format
            "symbol": symbol,
            "apikey": self.apikey,
            "exchange": "NASDAQ"
        }
        try: #Then we'll try to make the call to the API and return its formatted response as a JSON text
            # print(f'Calling FMP API at endpoint: {self.endpoint} with params: {params}')
            response=requests.get(self.endpoint, params=params)
            return response.json()
        except requests.exceptions.RequestException as e: # Should there be any errors, we'll print the error message and return an empty dictionary
            print(f'FMP API error: {e}')
            return {}
        
class StockQuote(FMP):
    def __init__(self):
        super().__init__(
            name="Stack Quote", # Name of the tool
            description="Get the latest stock quote for a given symbol from Stack Quote.", # Definition of the tool for our agents
            api="""{ "symbol": "AAPL"}""", # Parameter sample for the agent to use when it uses this class
            endPoint='https://financialmodelingprep.com/stable/quote' # It reads the endpoint from our .env file
        )

        
class StockPriceChange(FMP):
    def __init__(self):
        super().__init__(
            name="Stock Price Change", # Name of the tool
            description="Get the stock price change for a given symbol over the past.", # Definition of the tool for our agents
            api="""{ "symbol": "AAPL", "days": 7}""", # Parameter sample for the agent to use when it uses this class
            endPoint='https://financialmodelingprep.com/stable/stock-price-change' # It reads the endpoint from our .env file
        )
        
class IncomeStatement(FMP):
    def __init__(self):
        super().__init__(
            name="Income Statement", # Name of the tool
            description="Get the income statement for a given symbol from Financial Modeling Prep (FMP).", # Definition of the tool for our agents
            api="""{ "symbol": "AAPL"}""", # Parameter sample for the agent to use when it uses this class
            endPoint='https://financialmodelingprep.com/stable/income-statement' # It reads the endpoint from our .env file
        )
  
class FinancialScore(FMP):
    def __init__(self):
        super().__init__(
            name="Financial Score", # Name of the tool
            description="Get the financial score for a given symbol from Financial Modeling Prep (FMP).", # Definition of the tool for our agents
            api="""{ "symbol": "AAPL"}""" ,# Parameter sample for the agent to use when it uses this class
            endPoint='https://financialmodelingprep.com/stable/financial-scores' # It reads the endpoint from our .env file
        )
   
        
#We'll be using FinnHub as our News provider next
class FinancialNews(Tool): 
    def __init__(self):
        super().__init__(
            name="FinnHub News", # Name of the tool
            function=self.get_stock_quote_finnhub, # Pointing to the FinnHub function below as this class's own function
            description="Get the latest financial news for a given symbol from FinnHub.", # Definition of the tool for our agents
            api="""{ ""symbol": "AAPL"}""" # Parameter sample for the agent to use when it uses this class
        )
    def get_stock_quote_finnhub(self, symbol: str, step: str='') -> dict: # This is the function that pulls the news data using FinnHub
        FinnHubAPIKey = os.getenv("FINNHUB_API_KEY") # Gets the API key from our .env file
        # Next, we setup the client to perform calls:
        finn_client = finnhub.Client(api_key=FinnHubAPIKey)

        # Setting a time frame for the news, ending today and starting a week ago
        end_date = datetime.today().strftime("%Y-%m-%d")
        start_date = (datetime.today() - timedelta(days=7)).strftime("%Y-%m-%d")

        # Now, we call the API, returning the news in the already pre-formatted dictionary structure.
        try:
            news= finn_client.company_news(symbol, _from=start_date, to=end_date) # Calling the FINN API
            if len(news)==0:
                return {"message": f"No news found for symbol {symbol} from {start_date} to {end_date}."}
            top_news = sorted(news, key=lambda x: x['datetime'], reverse=True)[:5] # Pulls only the top 5 news
            top_news_formatted = []
            for item in top_news: # Iterates through each news item to pull the headline, summary, and date.
                top_news_formatted.append({
                    "headline": item.get("headline"),
                    "summary": item.get("summary"),
                    "datetime": datetime.fromtimestamp(item.get("datetime")).strftime("%Y-%m-%d %H:%M:%S")
                })
        
            return { # Returns the formatted news for the specified symbol
                "symbol": symbol,
                "news": top_news_formatted  
            }
    
        except Exception as e: # Should there be any errors, we'll print the error message and return an empty dictionary
            print(f'Finnhub.io API error: {e}')
            return {}

class RecommendationTrends(Tool):
    def __init__(self):
        super().__init__(
            name="FinnHub Recommendation Trends", # Name of the tool
            function=self.get_recommendation_trends, # Pointing to the FinnHub function below as this class's own function
            description="Get the recommendation trends for a given symbol from FinnHub.", # Definition of the tool for our agents
            api="""{ ""symbol": "AAPL"}""" # Parameter sample for the agent to use when this class
        )
    def get_recommendation_trends(self, symbol: str) -> dict:
        FinnHubAPIKey = os.getenv("FINNHUB_API_KEY") # Gets the API key from our .env file
        finn_client = finnhub.Client(api_key=FinnHubAPIKey)
        try:
            return finn_client.recommendation_trends(symbol)
        except Exception as e: # Should there be any errors, we'll print the error message and return an empty dictionary
            print(f'Finnhub.io API error: {e}')
            return {}
        
class EarningSurprise(Tool):
    def __init__(self):
        super().__init__(
            name="FinnHub Earning Surprise", # Name of the tool
            function=self.get_earning_surprise, # Pointing to the FinnHub function below as this class's own function
            description="Get the earning surprise for a given symbol from FinnHub.", # Definition of the tool for our agents
            api="""{ ""symbol": "AAPL"}""" # Parameter sample for the agent to use when this class
        )
    def get_earning_surprise(self, symbol: str) -> dict:
        FinnHubAPIKey = os.getenv("FINNHUB_API_KEY") # Gets the API key from our .env file
        finn_client = finnhub.Client(api_key=FinnHubAPIKey)
        try:
            return finn_client.company_earnings(symbol,limit=5)
        except Exception as e: # Should there be any errors, we'll print the error message and return an empty dictionary
            print(f'Finnhub.io API error: {e}')
            return {}

C:\Users\isral\.conda\envs\Transformers_3_10\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


### 2. PARSER
#### One of the workflow patterns our agents will do is routing, meaning our main agent will coordinate with subagents. To accomplish this communication, we need a "common language", which in this case will be JSON. This section defines the functions to implement the JSON parsing functionality.

In [2]:
# We'll define first a Parser abstract class
class Parser:
    def parse(self, response): # This is the placeholder of the default method for this class
        # Here's the returned value, which will be a dictionary with an Action value, and a list of dynamic parameters.
        return {"action": "FinalAnswer", "parameters": {}}
# Next, we'll define an XML parser, which inherits from our abstract class Parser.    
class XmlParser(Parser):
    def parse(self, response):
        # A parser that extracts XML tags from the response.
        # For example, it looks for <InvokeTool>{"symbol": "AAPL", "step": "financials"}</InvokeTool>
        # or <FinalAnswer>answer</FinalAnswer>.
        # Returns : a dict with action and parameters. Example:
        # {
        #    "action": "InvokeTool",
        #    "parameters": {
        #        "symbol": "AAPL",
        #        "step": "financials"
        #    }
        #}
        import re
        pattern = r'<(\w+)>(.*?)</\1>' # Defining the regular expression for XML structure
        matches = re.findall(pattern, response) # Identifying all matches of XML
        if matches: # When there are XML matches, we'll separate them and parse their contents
            action, content = matches[0]
            content = content.strip()
            contentJson = {}
            try:
                import json
                contentJson = json.loads(content) # Once parsed, we'll reformat them to JSON
            except:
                contentJson = {"content": content} # If the content is not valid, we'll return the error message with the invalid content text
                return {"action": action, "parameters": contentJson, "error": "Content is not valid JSON"}
            return {"action": action, "parameters": contentJson} # If it was valid, we return the parsed content in JSON format
        return {"action": "FinalAnswer", "parameters": {}} #If there wasn't any XML to begin with, we just return an empty list of parameters
    # Next, we have a specialized parsing for our agent's functionality that will interpret the actions in XML tags and encode them as a list of dictionaries
    def parse_all(self, response):
        import re
        pattern = r'<(\w+)>(.*?)</\1>' # Defining the regular expression for XML structure
        matches = re.findall(pattern, response) # Identifying all matches of XML
        results = [] # Preparing an empty array for the results
        for action, content in matches: # For each detected action (if any),
            content = content.strip()   # we'll parse its contents
            contentJson = {}
            try: # Then, we'll try to convert it to JSON format
                import json
                contentJson = json.loads(content)
            except: # Should any errors occur, we'll return the error message as part of the response
                contentJson = {"content": content}
                results.append({"action": action, "parameters": contentJson, "error": "Content is not valid JSON"})
                continue
            results.append({"action": action, "parameters": contentJson}) # If everything's fine, we'll return the parsed JSON content
        if not results:
            results.append({"action": "FinalAnswer", "parameters": {}}) # If there were no actions, we'll return an empty dictionary
        return results  
    
    def parseTags(self, response):
        '''Agent response parser to extract all TAGS.
            Returns a dictionary with tag names as keys and tag values as values.
        '''
        import re
        pattern = r'<(\w+)>(.*?)</\1>'
        matches = re.findall(pattern, response)
        result = {}
        for tag, value in matches:
                result[tag.lower()] = value.strip() 
        return result

### 3. MEMORY.
#### Another feature of our agent is learning, which means the agent must remember information as it gets prompted to refine their answers and keep getting more knowledgeable as it gets used. The functions that perform such learning are defined in this section.

In [3]:
import os
import pickle
# We're creating a class called MemorySystem with all the learning functionality
class MemorySystem:
    # This class stores insights and lessons from previous analyses to improve future runs.
    def __init__(self, memory_file='agent_memory.pkl'): # It will store the learned data into the specified file, or the default file name.
        self.memory_file = memory_file
        self.stock_insights = {}
        self.news_insights = {}
        self.load_memory()
    
    def load_memory(self): # Should there be a previous file in existence, it can load it using this function
        try:
            if os.path.exists(self.memory_file): # It will look for the file name specified in the instance of this class
                with open(self.memory_file, 'rb') as f: # If it exists, it will attempt to open it
                    memory_data = pickle.load(f) # Then, it will load the data into memory
                    self.stock_insights = memory_data.get('stock_insights', {}) # separating stock insights,
                    self.news_insights = memory_data.get('news_insights', {}) # market news insights,
            else: # Should there be no prior file, it will start fresh
                print("No memory file found. Starting with empty memory.")
        except Exception as e: # Should there be an error while loading the file, it will start fresh as well
            print(f"Error loading memory: {e}")
            print("Starting with empty memory.")
    
    def save_memory(self): # This method will save the memory in the file in a structured manner
        try:
            memory_data = {
                'stock_insights': self.stock_insights, # It will save all stock insights currently provided,
                'news_insights': self.news_insights # followed by news insights
            }
            with open(self.memory_file, 'wb') as f: # It will first open the file name specified in the instance of this class
                pickle.dump(memory_data, f) # and then write in it the contents of the memory_data dictionary
            print("Memory saved successfully.")
        except Exception as e:
            print(f"Error saving memory: {e}") # Should there be any errors saving, it will print out the error
    
    def add_stock_insight(self, symbol, insight, timestamp=None): # With this method, we'll add knowledge classified as stock insights
        if timestamp is None:
            timestamp = datetime.now().isoformat() # If no timestamp is specified, we'll initialize the current time stamp
        
        if symbol not in self.stock_insights: # If the current symbol (financial company) is not in previous insights, we'll add it
            self.stock_insights[symbol] = []
        
        self.stock_insights[symbol].append({ # Finally, we encode the insight with its timestamp in the stock_insights dictionary of this class
            'insight': insight,
            'timestamp': timestamp
        })
        self.save_memory() # And we save the memory right away
    
    def add_market_news(self,symbol, news_item, timestamp=None): # This method adds market news insights for a given symbol
        if timestamp is None:
            timestamp = datetime.now().isoformat() # If no timestamp is specified, we'll initialize the current time stamp

        if symbol not in self.news_insights: # If the current symbol (financial company) is not in previous insights, we'll add it
            self.news_insights[symbol] = []

        self.news_insights[symbol].append({ # Finally, we encode the news item with its timestamp in the news_insights dictionary of this class
            'news_item': news_item,
            'timestamp': timestamp
        })
        self.save_memory() # And we save the memory right away

    def get_stock_insights(self, symbol): # This method retrieves all stock insights for a given symbol
        results=self.stock_insights.get(symbol, [])
        if not results:
            print(f"No insights found for symbol {symbol}.")
            return []
        if results:
            filtered_results = []
            for result in results:
                # if the timestamp is older than 7 days, we can choose to ignore it
                timestamp = datetime.fromisoformat(result['timestamp'])
                if (datetime.now() - timestamp).days > 7:
                    continue
                filtered_results.append(result)
        return filtered_results

    def get_news_insights(self, symbol): # This method retrieves all market news insights for a given symbol
        results=self.news_insights.get(symbol, [])
        if not results:
            print(f"No news insights found for symbol {symbol}.")
            return []
        if results:
            filtered_results = []
            for result in results:
                # if the timestamp is older than 2 days, we can choose to ignore it
                timestamp = datetime.fromisoformat(result['timestamp'])
                if (datetime.now() - timestamp).days > 2:
                    continue
                filtered_results.append(result)
        return filtered_results

### 4. AGENTS
#### For our routing workflow, along with communication also comes specialization and tool usage: a team of agents that will collaborate, coordinated by the main orchestrator agent. That's what we'll define in this section.

In [13]:
#First, we'll initialize the Google GenAI and OpenAI
from google import genai
import openai
#Make sure to load the environmental variables
dotenv.load_dotenv(dotenv_path=".env")

import numpy as np
import pandas as pd
import yfinance as yf

class Agent: # This will be our base class for all our agents
    def __init__(self, name, role, system_prompt, model, generate_response, agents=None, tools=None, memory_system=None, parser=None, debug=0): # This is the initialization method of the Agent class
        self.name = name # Placeholder for the name of the tool
        self.model = model # Placeholder for the LLM model
        self.role = role # Placeholder for the role of this agent
        self.system_prompt = system_prompt # Placeholder for the system prompt that defines this agent
        self.memory_system = memory_system # Placeholder for the memory object for this agent - it could be None, so the agent would start without knowledge
        self.parser = parser  # Placeholder for API details when needed
        self.generate_response = generate_response # Placeholder for the generate response method
        self.agents = agents 
        self.tools = tools # Placeholder for the tools passed on to this agent, which should be a list
        self.conversation_history = [] # Initializing a blank conversation history
        self.max_history_length = 10 # Initializing a default max number of history length

        self.prompt_template = (
            "You are {agent_name}, an AI agent. Use the following tools as needed:\n"
            "{tools}\n"
            "Conversation history:\n"
            "{history}\n"
            "Current input: {input}\n"
            "Respond appropriately."
        )
        self.initialize_client() #Initializing the LLM client
        self.debug = debug #Setting the debug local variable, used to print certain validation statements when set to 1
    #We want our Agent class to support multiple LLMs, so this function will help initialize its internal client dynamically.
    def initialize_client(self):
        #For GPT models
        if "gpt" in self.model.lower(): 
            self.client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        #For Gemini models
        elif "gemini" in self.model.lower():
            self.client = genai.Client()
    def to_dict(self): # The structure of each class will always be a standard dictionary object that can be easily interpreted by the Agents
        return {
            "name": self.name,
            "description": self.description,
            "api": self.api
        }
    def register_tool(self, tool): #This function helps register tools that the agent will have access to.
        self.tools.append(tool) 
    def remember(self, message): #This function enables the agent to remember a message in its conversation history
        self.conversation_history.append(message)
        if len(self.conversation_history) > self.max_history_length:
            self.conversation_history.pop(0) # If the number of elements in the history exceeds the established limit, we'll pop the oldest one
    def call_llm(self, input_prompt): #This is the generic call to LLM that agents can use. They may have a different version if needs are unique
        try:
            #For GPT models
            if "gpt" in self.model.lower(): 
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=[
                        {"role": "system", "content": self.system_prompt},
                        {"role": "user", "content": input_prompt}
                    ],
                    max_tokens=300,
                    temperature=0.7
                )
                result = response.choices[0].message.content
            #For Gemini models
            elif "gemini" in self.model.lower():
                prompt = self.system_prompt
                prompt += "\n" + input_prompt
                response = self.client.models.generate_content(
                    model=self.model, contents=str(prompt)
                )
                result = response.text
            return result
        except Exception as e:
            if self.debug == 1:
                print(f" API failed for {self.name} using model '{self.model}': {e}")
            return f"Mock response from {self.name} with model '{self.model}': {input_prompt[:50]}..."
    def generate_response(self, **kwargs): # This is the placeholder of the generative function for the agent, which will receive a variable number of parameters
        if self.debug == 1:
            print(f"Invoking {self.name} generative response function with arguments {kwargs}")
        return self.generate_response(**kwargs) # Returning the results of the function

#### Next, let's define some Sub-agents that will inherit from the class above.
#### We'll start with the Market Research agent, capable of finding hard-financial data like stock quotes or market trends using some of the financial tools declared at the top.

In [5]:
class MarketResearchAgent(Agent):
    def __init__(self, model="gemini-2.5-flash", debug=0):
        name="Market Research Agent"
        model=model
        role="Market Research Agent specialized in financial data analysis and market trends"
        system_prompt=f"""You are a Market Research Agent specialized in financial data analysis and market trends.
         Your role is to assist users by providing accurate and up-to-date financial information, stock quotes, market trends, and insights based on the latest data available from various financial APIs and tools.

         Based on the data retrieved from the tools at your disposal, provide comprehensive answers to user queries related to stock performance, market analysis, and financial news.
        
        """
        self.memory_system=MemorySystem()
        super().__init__(name=name,system_prompt=system_prompt,model=model,generate_response=self.generate_response,role=role,agents=None,tools=None,memory_system=self.memory_system,parser=None, debug=debug) 
    
    def generate_response(self, **kwargs): # This is the generative function for the agent, which receives a prompt and returns the result from the LLM
        if self.debug == 1:
            print(f"Invoking {self.name} generative response function with arguments {kwargs}")
        input_prompt=kwargs.get("prompt",[])
        try:
            #For GPT models
            if "gpt" in self.model.lower(): 
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=[
                        {"role": "system", "content": self.system_prompt},
                        {"role": "user", "content": input_prompt}
                    ],
                    #messages=input_prompt,
                    max_tokens=300,
                    temperature=0.7
                )
                result = response.choices[0].message.content
            #For Gemini models
            elif "gemini" in self.model.lower():
                response = self.client.models.generate_content(
                    model=self.model, contents=str(input_prompt)
                )
                result = response.text
            return result
        except Exception as e:
            if self.debug == 1:
                print(f" API failed for {self.name} using model '{self.model}': {e}")
            return f"Mock response from {self.name} with model '{self.model}': {input_prompt[:50]}..."

    def getMarketSummary(self,symbol:str  ) -> str: # This function pulls market data for the specified financial symbol
        prompt=f"""Provide a comprehensive market summary for the stock symbol: {symbol}. 
                Include recent performance, key financial metrics, and any notable news or trends affecting the stock.
                Use data from Yahoo Finance, Financial Modeling Prep, and FinnHub to inform your summary.
                Format the response in a clear and concise manner suitable for a financial report."""
        insights = self.memory_system.get_stock_insights(symbol) # It tries to pull existing insights in memory for the specified symbol
        if insights:
            if self.debug == 1:
                print(f"Using cached insight for symbol {symbol}.")
            return insights[-1]['insight'] # If found, it will return those cached insights for faster response
        else: # If no insights are found for this symbol, the agent will use RAG functions to pull new insights
            tools_list=[FinancialScore(),IncomeStatement(),StockQuote(),StockPriceChange()] # Registering available RAG tools for this agent
            for tool in tools_list:
                tool_response=tool.invoke(symbol=symbol) # We call each RAG tool to pull data for the specified symbol
                prompt+=f"\nData from {tool.name}: {tool_response}" # We prepare the prompt with the data returned from the RAG tool
            response=self.generate_response(prompt=prompt) # We call the the LLM with the prompts from all tools to produce a response
            self.memory_system.add_stock_insight(symbol, response,timestamp=datetime.now().isoformat()) # Adding the insight to the memory system
        return response # Returning the LLM response
    
    def  processUserInput(self, user_input: str) -> str:
        tags=self.getEntities(user_input=user_input) # We'll attempt to get the financial symbols from the user's input (i.e. AAPL for Apple, etc)
        if "symbol" in tags:
            marketSummary=self.getMarketSummary(symbol=tags.get("symbol")) # Then, we'll pull financial data for each symbol
        prompt=f"""Based on the {marketSummary} Analyze the following user input
                and provide a short answer for the user query.
                Rules:
                - If the user input is related to stock performance, provide insights based on the market summary.
                - If the user input is unrelated to financial markets, respond with "I'm sorry, I can only assist with financial market-related queries."
                - Keep the response concise and relevant to the user's query.
                - Use a professional and informative tone suitable for financial discussions.
                - Limit the response to 150 words.

                User Input: "{user_input}"


                Answer:
                """,
        response=self.generate_response(prompt=prompt) # Passing the financial date to the LLM to generate a response
        return response

    def getEntities(self, user_input: str) -> str: # This function reviews the user input and pulls any Financial symbols for the mendioned companies
        prompt=f"""Determine entities the following user input related to financial markets and stock analysis:
                if the input contains Apple Inc, return SYMBOL as AAPL
                if the input contains Microsoft Corporation, return SYMBOL as MSFT
                User Input: "{user_input}
                Extracted Entities:
                    <SYMBOL>...</SYMBOL>
                    <EXCHANGE>...</EXCHANGE><INDUSTRY>...</INDUSTRY>  """
        response=self.generate_response(prompt=prompt) # Sends the prompt to the LLM to extract the symbols
        if self.debug == 1:
            print(f'Response: {response}')
        parser=XmlParser() # We use our XML parser to extract the symbols from the LLM response
        parsed_response=parser.parseTags(response)
        return parsed_response

#### Then, we'll define a Market Sentiment Agent, which will pull financial news using some of the financial tools above, and classify their sentiment, which will be helpful for the research and analsys of the company in question.

In [6]:
class MarketSentimentAgent(Agent):
    def __init__(self, model="gemini-2.5-flash", debug=0):
        name="Market News Sentiment Agent"
        model=model
        role="Market News Sentiment Agent specialized in financial news sentiment analysis"
        system_prompt=f"""You are a Market Sentiment Agent specialized in financial news sentiment analysis.
         Your role is to assist users by analyzing the sentiment of financial news articles and providing insights based on the emotional tone of the content.

         Based on the news data retrieved from FinnHub, provide comprehensive sentiment analysis to help users understand market mood and potential impacts on stock performance.
        
        """
        self.memory_system=MemorySystem()
        super().__init__(name=name,system_prompt=system_prompt,model=model,generate_response=self.generate_response,role=role,agents=None,tools=None,memory_system=self.memory_system,parser=None, debug=debug)
        
    def generate_response(self, **kwargs): # This is the placeholder of the generative function for the agent, which will receive a variable number of parameters
        if self.debug==1:
            print(f"Invoking {self.name} generative response function with arguments {kwargs}")
        input_prompt=kwargs.get("prompt",[])
        try:
            #For GPT models
            if "gpt" in self.model.lower(): 
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=input_prompt,
                    max_tokens=300,
                    temperature=0.7
                )
                result = response.choices[0].message.content
            #For Gemini models
            elif "gemini" in self.model.lower():
                response = self.client.models.generate_content(
                    model=self.model, contents=str(input_prompt)
                )
                result = response.text
            return result # Returning the response provided by the selected model
        except Exception as e:
            if self.debug==1:
                print(f" API failed for {self.name} using model '{self.model}': {e}")
            return f"Mock response from {self.name} with model '{self.model}': {input_prompt[:50]}..."
        
    def getNewsSummary(self,symbol:str  ) -> str:
            prompt=f"""Provide a comprehensive news summary for the stock symbol: {symbol}.
                    Include recent news articles, key events, and any notable trends affecting the stock.
                    Use data from FinnHub and other news sources to inform your summary.
                    Format the response in a clear and concise manner suitable for a financial report."""
            insights = self.memory_system.get_news_insights(symbol) # If there are any insights found in memory, we'll pull them here
            if insights:
                if self.debug==1:
                    print(f"Using cached insight for symbol {symbol}.")
                return insights[-1]['news_item'] # We'll return the insights if found.
            else: # If there are no insights, we'll call RAG tools to get new insights.
                tools_list=[FinancialNews(),RecommendationTrends(),EarningSurprise()] # Declaring the list of financial tools
                for tool in tools_list: # Pulling insights from each available tool
                    tool_response=tool.invoke(symbol=symbol)
                    prompt+=f"\nData from {tool.name}: {tool_response}"
                response=self.generate_response(prompt=prompt) # Pass the retrieved RAG data from tools to the LLM to generate a response
                self.memory_system.add_market_news(symbol, response,timestamp=datetime.now().isoformat()) # Storing the response in memory
            return response
        
    def processUserInput(self, user_input: str) -> str:
        if self.debug==1:
            print("-" * 50)
            print(f'{self.name}" received input: {user_input}')
            print("-" * 50)
        tags=self.getEntities(user_input=user_input) # We'll attempt to get the financial symbols from the user's input (i.e. AAPL for Apple, etc)
        if "symbol" in tags:
            newsSummary=self.getNewsSummary(symbol=tags.get("symbol")) # Then, we'll pull news for each symbol
        prompt=f"""Based on the {newsSummary} Analyze the following user input
                and provide a short answer for the user query.
                Rules:
                - If the user input is related to financial news sentiment, provide insights based on the news summary.
                - If the user input is unrelated to financial markets, respond with "I'm sorry, I can only assist with financial market-related queries."
                - Keep the response concise and relevant to the user's query.
                - Use a professional and informative tone suitable for financial discussions.
                - Limit the response to 150 words.

                User Input: "{user_input}"


                Answer:
                """,
        response=self.generate_response(prompt=prompt) # Passing these news to the LLM to generate a response
        return response
    def getEntities(self, user_input: str) -> str: # This function reviews the user input and pulls any Financial symbols for the mendioned companies
        prompt=f"""Determine entities the following user input related to financial markets and stock analysis:
                if the input contains Apple Inc, return SYMBOL as AAPL
                if the input contains Microsoft Corporation, return SYMBOL as MSFT
                User Input: "{user_input}
                Extracted Entities:
                    <SYMBOL>...</SYMBOL>
                    <EXCHANGE>...</EXCHANGE><INDUSTRY>...</INDUSTRY>  """
        response=self.generate_response(prompt=prompt) # Sends the prompt to the LLM to extract the symbols
        if self.debug == 1:
            print(f'Response: {response}')
        parser=XmlParser() # We use our XML parser to extract the symbols from the LLM response
        parsed_response=parser.parseTags(response) 
        return parsed_response

#### Then, we'll define a Writer agent in charge of providing a polished and structured answer with the researched data provided by the other agents.

In [7]:
class WriterAgent(Agent):
    # This agent takes the results of other agents (like news or market research) and creates a professional report that will be returned to the Orchestrator for the Final Response to the user.
    def __init__ (self, model="gpt-3.5-turbo", agents=None, memory_system=None, debug=0):
        super().__init__(
            name="Writer", #Name of the Writer class
            role="Writer Agent specialized in polished Financial Content", #Role of this class
            system_prompt=(
                "You are Writer, an AI agent part of an agents team. Your role is a professional "
                "financial report writer, capable of taking financial news "
                "or financial information provided by the Orchestrator and "
                "preparing a 2–3 paragraph report that provides a clear final "
                "answer to the user."
                "Here are some guidelines for you:"
                "Start your answers giving a positive message like 'Great question', 'Excellent question', or similar."
                "Focus on answering the user's question."
                "When recommendations are requested, only provide guidance and highlight pros and cons."
                "The news or financial information you're receiving came from other agents in the team, so never refer to it as 'the data provided'."
            ),            
            model = model,
            generate_response = self.generate_response,
            memory_system = memory_system,
            debug=debug
        )
    def generate_response(self, input_prompt): # Calling the default LLM function from the base class
        result = self.call_llm(input_prompt)
        return result
    def processUserInput(self, input_prompt: str) -> str: # This function receives the request from the orchestrator and returns an LLM result.
        prompt=input_prompt
        response=self.generate_response(input_prompt=prompt)
        return response    

### 5. MAIN ORCHESTRATOR AGENT.
#### This is the section where we define the orchestrator agent, which performs the interpretation of the user's prompt, prepares a plan, calls the subagents as needed, and prepares the final answer to the user.

In [8]:
class OrchestratorAgent(Agent):
    def __init__(self, model, agents=None, memory=None, parser=None, debug=0):
        self.agents = agents
        self.agents_description = ""
        for agent in self.agents:
            self.agents_description += f'\n- {agent.name}: {agent.role}'

        system_prompt = f'''
          You are the leading AI agent for the following team of agents:
            {self.agents_description}

            You do not generate a response directly to the user, but instead you'll coordinate the agents team by generating a list of tasks for them to do following the Agent Usage guidelines.
            
            ### Agent Usage Guidelines:

            1. Do not respond to the current input directly. Instead, create a plan to call the research agents in your team to pull the necessary data.
            2. Convert that plan into a list of calls for your specialized agents (except for the Writer Agent) using an XML structure with the tag "<SpecializedAgent>" in the following format:
            <SpecializedAgent>{{"agentName": "Market Research Agent", "user_input": "Your specific query here"}}</SpecializedAgent>
            3. The writer agent will be called separately to finalize the response. Exclude from your thinking process.
            
            ### Other TAGs you can include in your plan:
            -  For thinking, you must wrap your thoughts in <Thought> and </Thought> tags.
            -  For final answers, you must wrap your answer in <FinalAnswer> and </FinalAnswer> tags.
            -  If you need users to provide more information, you must wrap your request in <RequestMoreInfo> and </RequestMoreInfo> tags.
           
            ### Instructions for using the tools:
            You should only use the information returned by the Agents listed above, never try to get information independently.
        '''
        system_prompt = system_prompt.replace("{", "{{").replace("}", "}}")
        # We also need to declare a parser
        if parser==None:
            parser = XmlParser()

        super().__init__(
            name = "Orchestrator Agent", #Name of the Orchestrator class
            role = "Orchestrator Agent that manages tool usage and conversation flow",
            system_prompt = system_prompt,
            model = model,
            generate_response = self.generate_response,
            memory_system = memory,
            agents = agents,
            parser = parser,
            debug = debug # Storing the variable debug, used for printing messages when set to 1
        )
        self.conversation_history = []
        self.prompt_template = (
            f"{self.system_prompt}\n"
            "Conversation history:\n"
            "{history}\n"
            "Current input: {input}\n"
        )
        self.parser = parser
        self.initialize_client()

    def remember(self, message): # This function will keep track of the message in the conversation history
        self.conversation_history.append(message)
        if len(self.conversation_history) > self.max_history_length:
            self.conversation_history.pop(0) #It will also remove the last element if the history exceeds the max length
    
    def generate_response(self, input_prompt): # This function calls the LLM with the specified prompt 
        history_text = "\n".join(self.conversation_history)
        response = ""
        prompt = self.prompt_template.format(
            history=history_text,
            input=input_prompt
        )
        if self.debug==1:
            print(f"Orchestrator Prompt: {prompt}")
        try:
            #For GPT models
            if "gpt" in self.model.lower(): 
                response = self.client.chat.completions.create(
                    model=self.model,
                    #messages=input_prompt,
                    messages=[
                        {"role": "system", "content": prompt}
                    ],
                    max_tokens=300,
                    temperature=0.7
                )
                result = response.choices[0].message.content
            #For Gemini models
            elif "gemini" in self.model.lower():
                response = self.client.models.generate_content(
                    model=self.model, contents=str(input_prompt)
                )
                result = response.text
            self.remember(f"User: {input_prompt}")
            self.remember(f"{self.name}: {response}")
            return result
        except Exception as e:
            if self.debug==1:
                print(f" API failed for {self.name} using model '{self.model}': {e}")
            return f"Mock response from {self.name} with model '{self.model}': {input_prompt[:50]}..."
        return response
    

    def get_specialist_opinion(self, agentName, user_input):
        # Agent Orchestrator can call other agents to get their opinion on specific user inputs.
        MyAgentsTeam = {MyMarketResearcher, MyNewsResearcher, MyWriter}
        for agent in self.agents:
            if agent.name == agentName: # When we find the agent referred in the parameters,
                return agent.processUserInput(user_input) # we  proceed sending the request to that agent
        return f"Agent {agentName} not found." # If we reach this line, it means the agent in the parameters wasn't found.
        
    
    def reAct(self, user_input:str)-> str:
        # Here is the the logic to parse the response for Agents usage
        # and store the results.
        parsed_response = ""
        #Preparing a temporary repository for agent responses
        temp_agent_response = ""
        #We also initialize the content variable we'll pass to the writer
        content_for_writer = f'Current user prompt: {user_input}'
        if self.parser and self.agents:
            response = self.generate_response(user_input) # We ask the LLM to pur together the plan of actions, and store it raw in the response variable.
            parsed_response = self.parser.parse_all(response) ## parsed response is a dict {"InvokeTool": "tool_name", "parameters": {...}} or {"FinalAnswer": "answer"} or {"RequestMoreInfo": "info"}
            if self.debug==1: # If debugging, we'll print the above variables
                print("*" * 50)
                print(f'Raw actions from Orchestrator: {response}')
                print("*" * 50)
                print("*" * 50)
                print(f'Actions list from Orchestrator: {parsed_response}')
                print("*" * 50)
            system_message = f"System: {response}"
            self.remember(system_message) #We store the raw plan in the conversation history
            self.conversation_history.append(system_message)
            '''
            parsed_response= {
            "action": "InvokeTool",
            "parameters": {
                "symbol": "AAPL",
                "step": "financials"
                }
            }
            '''
            # Next, we'll loop through all the actions in the plan to execute one at a time.
            for plan_item in parsed_response:
                action = plan_item.get("action") # We get the next action on the list
                if self.debug==1:
                    print(f"Orchestrator Action: {action}")
                if action == "SpecializedAgent": # When the action is to call another agent, we'll prepare the prompt for that agent here
                    agent_name = plan_item["parameters"].get("agentName") # Selecting the agent name
                    user_input_for_agent = plan_item["parameters"].get("user_input") # Getting the prompt in the plan for that agent
                    if self.debug==1:
                        print("-" * 50)
                        print(f'Orchestrator calling {agent_name} with prompt "{user_input_for_agent}"')
                        print("-" * 50)
                    agent_response = self.get_specialist_opinion(agent_name, user_input_for_agent) # Getting the response from that agent
                    temp_agent_response = f"Agent {agent_name} Response: {agent_response}" # Preparing the prompt for the Writer agent by storing each agent's response
                    self.remember(temp_agent_response) # Adding each agent's response to memory
                    self.conversation_history.append(temp_agent_response) 
                    content_for_writer += f'\n\n{temp_agent_response}' # Attaching each agent's response to the content for the Writer agent
                elif action == "FinalAnswer" or action == "RequestMoreInfo" or action == "NeedApproval": # If it's any of these options, we need user input
                    print(f"Orchestrator Final Response: {response}")
                    return parsed_response.get("content")
                elif action == "Thought": # If it's a thought, we can ignore it on this loop
                    continue
                else: # Any other actions would be an error.
                    return f"I'm not sure how to proceed. Could you please clarify? - selected action: {action}"
            #Once the loop of actions is completed, we'll pass the information gathered by all research agents down to our writer
            response = self.get_specialist_opinion('Writer', content_for_writer) # Then, we'll return the response from the Writer as the Final Answer
        else:
            parsed_response = "Error: no parser or sub agents found!"
            print('Parser:')
            print(self.parser)
            print('Agents:')
            print(self.agents)
            response = parsed_response
        return response

### 6. DEMO.
#### This is the final section, which contains the implementation of the entire system using all elements above for a quick demonstration.
#### We'll start by declaring our team of agents.

In [9]:
#We can use "gpt-3.5-turbo" or "gemini-2.5-flash", #Uncomment if using Google GenAI

#Let's declare our sub agent instances first.
MyMarketResearcher = MarketResearchAgent(model="gemini-2.5-flash")
MyNewsResearcher = MarketSentimentAgent(model="gemini-2.5-flash")
MyWriter = WriterAgent(model="gemini-2.5-flash")

#We put all of our sub agents together as a list of objects
MyAgentsTeam = {MyMarketResearcher, MyNewsResearcher, MyWriter}

#Now, we declare our main orchestrator agent instance.
MyOrchestrator = OrchestratorAgent(model="gpt-3.5-turbo", agents=MyAgentsTeam)

#### Next, we'll define a visual interface to give the tool a "chatbot" look and feel. We decided to call it <b>"CapitalMind Financial Assistant"</b>.

In [10]:
# Import libraries for beautiful HTML display in Jupyter notebook
from IPython.display import HTML, display, clear_output
from datetime import datetime
import time

#### Now we'll define the visual style of the chat interface:

In [11]:
# CSS styles for beautiful chatbot interface
chatbot_css = """
<style>
.chat-container {
    max-width: 800px;
    margin: 0 auto;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    border-radius: 20px;
    padding: 20px;
    box-shadow: 0 10px 30px rgba(0,0,0,0.3);
}

.chat-header {
    text-align: center;
    color: white;
    font-size: 24px;
    font-weight: bold;
    margin-bottom: 20px;
    text-shadow: 2px 2px 4px rgba(0,0,0,0.5);
}

.message {
    margin: 15px 0;
    display: flex;
    align-items: flex-start;
}

.user-message {
    justify-content: flex-end;
}

.bot-message {
    justify-content: flex-start;
}

.message-bubble {
    max-width: 70%;
    padding: 15px 20px;
    border-radius: 20px;
    box-shadow: 0 4px 12px rgba(0,0,0,0.15);
    position: relative;
    line-height: 1.4;
}

.user-bubble {
    background: linear-gradient(135deg, #36d1dc 0%, #5b86e5 100%);
    color: white;
    margin-left: 30px;
}

.bot-bubble {
    background: white;
    color: #333;
    margin-right: 30px;
    border: 1px solid #e0e0e0;
}

.message-avatar {
    width: 40px;
    height: 40px;
    border-radius: 50%;
    margin: 0 10px;
    display: flex;
    align-items: center;
    justify-content: center;
    font-weight: bold;
    color: white;
    font-size: 16px;
}

.user-avatar {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
}

.bot-avatar {
    background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
}

.timestamp {
    font-size: 11px;
    color: rgba(255,255,255,0.7);
    margin-top: 5px;
    text-align: right;
}

.bot-timestamp {
    color: #999;
    text-align: left;
}

.typing-indicator {
    display: flex;
    align-items: center;
    justify-content: flex-start;
    margin: 15px 0;
}

.typing-bubble {
    background: white;
    padding: 15px 20px;
    border-radius: 20px;
    margin-right: 30px;
    margin-left: 50px;
    box-shadow: 0 4px 12px rgba(0,0,0,0.15);
}

.typing-dots {
    display: flex;
    gap: 4px;
}

.dot {
    width: 8px;
    height: 8px;
    background: #999;
    border-radius: 50%;
    animation: typing 1.4s infinite ease-in-out;
}

.dot:nth-child(1) { animation-delay: -0.32s; }
.dot:nth-child(2) { animation-delay: -0.16s; }

@keyframes typing {
    0%, 80%, 100% { transform: scale(0.8); opacity: 0.5; }
    40% { transform: scale(1); opacity: 1; }
}

.input-container {
    margin-top: 20px;
    text-align: center;
}

.status-message {
    text-align: center;
    color: white;
    font-style: italic;
    margin: 10px 0;
    opacity: 0.8;
}
</style>
"""

# Helper functions for chatbot display
def display_chat_header():
    # Display the chatbot header
    header_html = f"""
    {chatbot_css}
    <div class="chat-container">
        <div class="chat-header">
            💰 CapitalMind Financial Assistant 🤖
        </div>
        <div class="status-message">
            Welcome! Ask me anything about financial markets and stocks.
        </div>
    </div>
    """
    display(HTML(header_html))

def display_user_message(message):
    # Display user message with chat-bot styling
    timestamp = datetime.now().strftime("%H:%M")
    user_html = f"""
    <div class="chat-container">
        <div class="message user-message">
            <div class="message-bubble user-bubble">
                {message}
                <div class="timestamp">{timestamp}</div>
            </div>
            <div class="message-avatar user-avatar">👤</div>
        </div>
    </div>
    """
    display(HTML(user_html))

def display_typing_indicator():
    # Display typing indicator (while the Agents are thinking)
    typing_html = f"""
    <div class="chat-container">
        <div class="typing-indicator">
            <div class="message-avatar bot-avatar">🤖</div>
            <div class="typing-bubble">
                <div class="typing-dots">
                    <div class="dot"></div>
                    <div class="dot"></div>
                    <div class="dot"></div>
                </div>
            </div>
        </div>
    </div>
    """
    display(HTML(typing_html))

def display_bot_message(message):
    # Display bot response with chat-bot styling
    timestamp = datetime.now().strftime("%H:%M")
    # Format the message with line breaks for better readability
    formatted_message = message.replace('\n', '<br>')
    
    bot_html = f"""
    <div class="chat-container">
        <div class="message bot-message">
            <div class="message-avatar bot-avatar">🤖</div>
            <div class="message-bubble bot-bubble">
                {formatted_message}
                <div class="timestamp bot-timestamp">{timestamp}</div>
            </div>
        </div>
    </div>
    """
    display(HTML(bot_html))

def display_status_message(message):
    # Display status message
    status_html = f"""
    <div class="chat-container">
        <div class="status-message">
            {message}
        </div>
    </div>
    """
    display(HTML(status_html))

def clear_typing_indicator():
    # Clear the typing indicator in preparation to display the actual reply
    clear_output(wait=True)

#### Let's see CapitalMind in action!!

In [12]:
#Displaying the header and launching the chat bot.
display_chat_header()

## Create a conversation loop with user input and orchestrator response
conversation_history = []
#Defining an infinite loop that will allow the user to have a conversation (until the user types the command "exit")
while True:
    try:
        user_input = input("💬 Your question: ")
        
        # Display user message with beautiful styling
        display_user_message(user_input)
        conversation_history.append(f"User: {user_input}")

        # Evaluating if the user entered the exit command first, in which case the conversation is over.
        if user_input.lower() == 'exit':
            display_status_message("👋 Exiting the financial market assistant. Goodbye!")
            break
        
        # Show typing indicator while processing
        display_typing_indicator()
        time.sleep(1)  # Brief pause for realistic effect
        
        # Get response from orchestrator
        response = MyOrchestrator.reAct(user_input)  # Pass just the user input, not the full history
        conversation_history.append(f"Orchestrator: {response}") # Capturing the Agents' response in the conversatio history
        
        # Clear typing indicator and display bot response
        clear_output(wait=True) #Clearing the output for next conversation
        display_chat_header() #Displaying the header again (restarting the conversation)
        
        # Redisplay recent conversation
        recent_history = conversation_history[-6:]  # Show last 3 exchanges
        for i in range(0, len(recent_history), 2):
            if i < len(recent_history):
                # Display user message
                user_msg = recent_history[i].replace("User: ", "")
                display_user_message(user_msg)
            if i + 1 < len(recent_history):
                # Display bot message
                bot_msg = recent_history[i + 1].replace("Orchestrator: ", "")
                display_bot_message(bot_msg)
        
    except KeyboardInterrupt:
        display_status_message("👋 Chat interrupted. Goodbye!")
        break
    except Exception as e:
        display_bot_message(f"❌ Sorry, I encountered an error: {str(e)}")
        display_status_message("Please try asking your question again.")
    

💬 Your question:  exit


## CONCLUSION.
#### The usage of Agentic AI systems is significantly more powerful than other generative AI techniques because it lets us specialize different tasks, separating the planning from the research and the response writing, providing flexibility to the complexity of the user's dynamic inquiry.
#### From the programming perspective, it also provides modularity, explainability, and transparency. This allows for flexibility as well in terms of expansion or upgrade of the skills of the solution, as well as making the maintenance of the tools more easily.
#### This financial exercise was great to understand the full process of designing and implementing an Agentic AI system.